# **Machine Learning para Business Intelligence** 
#### Profesor: Santiago Neira

## Clase 2. Preprocesamiento de datos, estructuración y visualización

In [ ]:
#pip install plotly

In [7]:
#pip install scikit-learn

In [8]:
#pip install pandas


In [9]:
#pip install numpy

In [10]:
# Importamos las librerías de la clase
import os
import pandas as pd
import numpy as np

In [11]:
# Definimos el directorio
os.getcwd()

'c:\\repos\\ML_4_BI\\Clase 2\\Scripts'

In [12]:
os.listdir("../Datos/")

['GOOG.csv', 'wine-quality.csv', 'winemag-data-130k-v2.csv']

In [13]:
os.chdir("../Datos/")

In [14]:
os.getcwd()

'c:\\repos\\ML_4_BI\\Clase 2\\Datos'

In [15]:
# Importamos los datos
vinos = pd.read_csv('winemag-data-130k-v2.csv',index_col=0)

In [16]:
# Importamos los datos
calidad_vinos = pd.read_csv('wine-quality.csv')

In [ ]:
# Veamos qué columnas tiene el dataframe
vinos.columns

0. Ciudad del Vino
1. Descripcion
2. Viñedo de donde provienne las uvas
3. Puntaje 1-100
4. Costo botella
5. Provincia o estado del vino
6. Desagregacion de ubicacion 1
7. Desagregacion de ubicacion 2
8. Nombre del catador
9. Twitter del catador
10. Titulo de la reseña
11. Tipo de uva para hacer el vino
12. La bodega del vino

In [ ]:
vinos.head()

In [ ]:
# Explore qué tipo de dato es el puntaje
vinos['points'].dtype

In [ ]:
# Explore qué tipos de datos son las diferentes columnas.
vinos.dtypes

In [ ]:
# ¿Cuál es la estructura del dataframe?
vinos.shape

In [ ]:
# Informacion de la base de datos (todo lo anterior en una sola vista)
vinos.info()

Para transformar una varibale a un tipo deseado se usa:
- Para datos discretos: ``astype(int)``
- Para datos continuos: ``astype(float)``
- Para datos categóricos: ``astype('category')``
- Para datos de texto: ``astype(str)``
- Entre otros...

In [ ]:
# Veamos que tipo de dato es country
vinos['country'].dtype

In [ ]:
# Veamos qué países aparecen en la columna 'country'
vinos['country'].unique()

In [ ]:
# Convierta la variable país en una variable categórica.
vinos['country'] = vinos['country'].astype('category')

In [ ]:
# Verifique que se hizo correctamente
vinos['country'].dtype

In [ ]:
vinos['country']

### Construcción de variables
1. Genere categorías de calificación de los vinos según el puntaje y usando la función ``pd.cut()``.
2. Convierta esta nueva variable en variables categóricas. No tenga en cuenta una de las categorías (¿Por Qué?).
3. Construya una variable booleana que capture si el vino es recomendado cuando pasa cierto umbral de puntaje.

In [ ]:
# Explorar la distribución de la variable y escoger puntos de corte adecuados.
vinos['points'].describe()

In [ ]:
# Queremos un punto de corte para definir un vino excelente!
q25 = np.quantile(vinos['points'], .25) # Percentil 25
q75 = np.quantile(vinos['points'], .50) # Percentil 75
x = np.floor(0.5*(q75 + vinos.points.max())) # Un punto entre el percentil 75 y el máximo

In [ ]:
# Un vino por encima de 92 resulta siendo excelente
x

In [ ]:
# Genere las categorías con pd.cut()
cats = ['malo', 'regular', 'bueno', 'excelente']
cortes = [vinos.points.min(), q25, q75, x, vinos.points.max()]
cortes

In [ ]:
vinos['class'] = pd.cut(vinos['points'], 
                        cortes, 
                        labels = cats)

In [ ]:
vinos['points'].tail()

In [ ]:
vinos['class'].tail()

In [ ]:
# Explore el tipo de datos de su variable creada
vinos['class'].dtype

In [ ]:
vinos.head()

In [ ]:
# Genere variable booleana de recomendación (puntaje mayor o igual a 92)
vinos['recommend'] = vinos['points'] >= 92

In [ ]:
vinos['recommend'].head()

In [ ]:
# proporcion de vinos que si son recomendados usando la variable recomendado
vinos['recommend'].mean()

In [ ]:
vinos['recommend'].value_counts()

## Estandarización

In [ ]:
# Dentro de los algoritmos que vamos a estar evaluando, en varios casos necesitaremos estandarizar las variables.
# Para esto, veamos rápidamente dos forma

## Diferencia entre MinMaxScaler y StandardScaler

### Conceptos Básicos

#### MinMaxScaler
- **Fórmula**: `X_scaled = (X - X_min) / (X_max - X_min)`
- **Rango resultante**: Datos escalados típicamente entre 0 y 1
- **Comportamiento**: Escala los datos basándose en los valores mínimos y máximos

#### StandardScaler
- **Fórmula**: `X_scaled = (X - mean) / std_deviation`
- **Rango resultante**: Datos con media 0 y desviación estándar 1
- **Comportamiento**: Centra los datos alrededor de 0 y escala según la varianza

### Comparación Visual
#### Datos originales sin
![Datos originales sin escalar](https://scikit-learn.org/stable/_images/sphx_glr_plot_all_scaling_001.png)

#### Datos después de aplicar StandardScaler:
![Datos con StandardScaler](https://scikit-learn.org/stable/_images/sphx_glr_plot_all_scaling_002.png)

#### Datos después de aplicar MinMaxScaler:
![Datos con MinMaxScaler](https://scikit-learn.org/stable/_images/sphx_glr_plot_all_scaling_003.png)

### Principales Diferencias

| Característica | MinMaxScaler | StandardScaler |
|----------------|--------------|----------------|
| Distribución de datos | Preserva la forma de la distribución original | Preserva la forma pero centra en 0 |
| Rango de valores | Entre 0 y 1 (por defecto) | No tiene límites definidos (generalmente entre -3 y 3) |
| Sensibilidad a outliers | Muy sensible a valores extremos | Sensible a outliers |
| Casos de uso ideales | Cuando se necesitan valores acotados (ej. redes neuronales) | Algoritmos que asumen distribución normal |
| Transformación | Lineal, basada en min/max | Lineal, basada en media/desviación |

### Cuándo Usar Cada Uno

#### Usar MinMaxScaler cuando:
- Los datos no siguen una distribución normal
- Se necesitan valores entre 0 y 1 (como entrada a una red neuronal)
- Los límites superior e inferior son bien conocidos
- Se trabaja con datos dispersos (sparse)
- Se usan algoritmos basados en distancias como K-Nearest Neighbors

#### Usar StandardScaler cuando:
- Los datos siguen aproximadamente una distribución normal
- Se usan algoritmos que asumen datos normalizados (regresión logística, SVM)
- Se desea dar el mismo peso a todas las características
- Se trabaja con algoritmos de gradiente descendente

### Efectos en Presencia de Outliers

- **MinMaxScaler**: Los outliers comprimen el resto de valores en un rango muy estrecho
- **StandardScaler**: Los outliers afectan el cálculo de la media y desviación estándar, pero tienen menos impacto en la escala general

### Recomendación

Lo mejor es probar ambos métodos de escalamiento y evaluar el rendimiento del modelo con cada uno. Si no estás seguro de cuál usar, comienza con StandardScaler para algoritmos basados en gradientes y MinMaxScaler para redes neuronales o algoritmos basados en distancia.

In [ ]:
calidad_vinos

In [ ]:
## Hay alguna forma de combinar los dos dataframes? Sí! Ambas columnas tienen el mismo título (identificador)

In [ ]:
info_vinos = calidad_vinos.merge(vinos,on='title',how='left').reset_index(drop=True)

In [ ]:
info_vinos.head(10)

In [ ]:
info_vinos.columns

Solo estandarizamos las variables numéricas

In [ ]:
vars_estandarizar = ['fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol',
       'quality','points']

In [ ]:
info_vinos[vars_estandarizar]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

estandarizacion_min_max = MinMaxScaler()
estandarizacion_min_max.fit(info_vinos[vars_estandarizar])

In [ ]:
estandarizacion_min_max.transform(info_vinos[vars_estandarizar])

In [ ]:
info_vinos_estandarizado = info_vinos.copy()
info_vinos_estandarizado.loc[:,vars_estandarizar] = estandarizacion_min_max.transform(info_vinos[vars_estandarizar])

In [ ]:
info_vinos_estandarizado[vars_estandarizar]

In [ ]:
(0.045-info_vinos.chlorides.min())/(info_vinos.chlorides.max()-info_vinos.chlorides.min())

Estandarización con la normal

In [ ]:
from sklearn.preprocessing import StandardScaler

estandarizacion_normal = StandardScaler()
estandarizacion_normal.fit(info_vinos[vars_estandarizar])

In [ ]:
info_vinos.loc[:,vars_estandarizar] = estandarizacion_normal.transform(info_vinos[vars_estandarizar])

In [ ]:
info_vinos.loc[:,vars_estandarizar]

In [ ]:
info_vinos_estandarizado_normal=pd.DataFrame(estandarizacion_normal.inverse_transform(info_vinos.loc[:,vars_estandarizar]),columns=vars_estandarizar)
info_vinos_estandarizado_normal

In [ ]:
# Veamos un ejemplo de diferencias entre los dos tipos de estandarización
info_vinos_estandarizado.loc[:,['fixed acidity','volatile acidity','citric acid']].describe()


In [ ]:
info_vinos_estandarizado_normal.loc[:,['fixed acidity','volatile acidity','citric acid']].describe()

## ¿Cómo lidiar con los NAs o missing values?
Primero vamos a identificar aquellas observaciones que son explícitamente NA

In [ ]:
# Identifique qué observaciones no cuentan con información de su precio
info_vinos['fixed acidity'].isna().head()

In [ ]:
info_vinos.isna().sum()

In [ ]:
np.round(100*info_vinos.isna().mean(), 2)

In [ ]:
# Opción 1: eliminar las observaciones sin información del precio
vinos1 = info_vinos[info_vinos['fixed acidity'].notna()]

In [ ]:
vinos1 = info_vinos[~info_vinos['fixed acidity'].isna()]

In [ ]:
vinos1 = info_vinos.dropna(subset = ['fixed acidity'])

In [ ]:
vinos1.shape

In [ ]:
# ¿Cuántas observaciones perdimos?
vinos1.shape[0] - info_vinos.shape[0]

In [ ]:
# ¿Qué proporción de observaciones perdimos?
vinos1.shape[0]/info_vinos.shape[0] - 1

In [ ]:
# Opción 2: Asignarles el precio promedio o mediano de las observaciones con esta información.
vinos2 = info_vinos.copy()
valor_promedio = vinos2['fixed acidity'].mean()
valor_promedio

In [ ]:
vinos2['fixed acidity'] = vinos2['fixed acidity'].fillna(vinos2['fixed acidity'].mean())
vinos2['fixed acidity'].head()

In [ ]:
info_vinos.dtypes

In [ ]:
# Opción 3: Asignandole la media/mediana de algún grupo relevante
tabla_imputacion = info_vinos.groupby("class")['fixed acidity'].median().reset_index()
tabla_imputacion

In [ ]:
diccionario_imputacion = dict(tabla_imputacion.values)
diccionario_imputacion

In [ ]:
# A cada clase se le está asignando un precio según el diccionario
valores_imputacion = info_vinos["class"].map(diccionario_imputacion)
valores_imputacion = np.array(valores_imputacion)
valores_imputacion

In [ ]:
filtro = info_vinos['fixed acidity'].isna()
filtro

In [ ]:
vinos3 = info_vinos.copy()

In [ ]:
vinos3.loc[filtro, "fixed acidity"] = valores_imputacion[filtro]

In [ ]:
vinos3.loc[filtro, ["fixed acidity", "class"]]

In [ ]:
# ¿Son los NA at random? ¿O siguen algún patrón?
np.round(100*vinos3["class"].value_counts(normalize = True), 2)

In [ ]:
np.round(100*vinos3.loc[filtro, ["price", "class"]].value_counts(normalize = True), 2)

In [ ]:
diccionario_imputacion

In [ ]:
# Comparemos los métodos 2, 3 
info_vinos.shape[0] == vinos2.shape[0] == vinos3.shape[0] 

In [ ]:
vinos2 = vinos2.rename(columns = {"fixed acidity": "acidity_media"})
vinos3 = vinos3.rename(columns = {"fixed acidity": "acidity_categoria"})

In [ ]:
imputar_media = vinos2[["acidity_media"]]
imputar_categoria = vinos3[["acidity_categoria"]]

In [ ]:
vinos_comparacion = pd.concat([info_vinos[['country', 'province', 'points', "fixed acidity"]], imputar_media, imputar_categoria], axis = 1)

In [ ]:
vinos_comparacion

In [ ]:
vinos_comparacion.loc[vinos_comparacion["fixed acidity"].isna(), ]

## Visualizaciones

In [ ]:
# Cargue las información del precio de la acción de Google
google = pd.read_csv('GOOG.csv', header=None, delimiter=',') # No tienen titulo 
google

In [ ]:
google.columns = ['Fecha', 'Precio'] # Asigne titulos
google.head()

In [ ]:
google.dtypes

In [ ]:
google['Fecha'] = pd.to_datetime(google['Fecha'], format="%d-%m-%Y")
google.head()

In [ ]:
google.dtypes

In [ ]:
#pip install matplotlib

In [ ]:
#pip install seaborn

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# https://www.python-graph-gallery.com/

In [ ]:
fig, ax = plt.subplots(figsize = (20, 6))
sns.lineplot(data = google, x = "Fecha", y = "Precio", ax = ax, color = "darkblue")
sns.scatterplot(data = google, x = "Fecha", y = "Precio", ax = ax, color = "darkblue")
ax.yaxis.set_major_formatter("${x:,.0f}")
plt.title("Precio de la acción de Google")

In [ ]:
import plotly.express as px

In [ ]:
google.isna().mean()

In [ ]:
#pip install nbformat>=4.2.0

In [ ]:
fig = px.line(data_frame = google, x = "Fecha", y = "Precio", title = "Precio de la acción de Google",
              markers = True)
fig.update_traces(line_color = 'darkblue', line_width = 2)
fig.show()

In [ ]:
# Número de vinos por país
paises = vinos.groupby('country').size().sort_values(ascending = False).reset_index(name = "Cantidad")
paises.head()

In [ ]:
fig, ax = plt.subplots(figsize = (20, 7))
sns.barplot(data = paises, x = "Cantidad", y = "country", order = paises["country"], ax = ax)
plt.xlabel("Cantidad")
plt.ylabel("País")

In [ ]:
paises["Porcentaje"] = paises["Cantidad"]/paises["Cantidad"].sum()
paises["Porcentaje acumulado"] = paises["Porcentaje"].cumsum()
paises

In [ ]:
paises.loc[paises["Cantidad"] < 1000, "country"] = "Otro"

In [ ]:
paises["country"] = paises["country"].astype(str)
paises.loc[paises["Cantidad"] < 1000, "country"] = "Otro"

In [ ]:
paises2 = paises.groupby("country")[["Cantidad", "Porcentaje"]].sum().reset_index().sort_values("Cantidad", ascending = False)

In [ ]:
paises2

In [ ]:
fig, ax = plt.subplots(figsize = (20, 7))
sns.barplot(data = paises2, x = "Cantidad", y = "country", order = paises2["country"], ax = ax,
            palette = "RdYlGn_r",hue="country")
plt.xlabel("Cantidad de vinos")
plt.ylabel("País")
ax.xaxis.set_major_formatter("{x:,.0f}")

# Anotación del valor de cada barra
total = paises2.Cantidad.sum()
for barra in ax.patches:
    h, w = barra.get_y(), barra.get_width()
    ax.text(x = w + 1500, y = h + 0.38, s = "{:.2%}".format(w/total), # "{:,.0f}".format(w)
            ha = 'center',
            va = 'center')

In [ ]:
fig, ax = plt.subplots(figsize = (20, 7))
sns.barplot(data = paises2, x = "Cantidad", y = "country", order = paises2["country"], ax = ax,
            palette = list(np.where(paises2.country == "Otro", "Red", "Grey")),hue='country')
plt.xlabel("Cantidad de vinos")
plt.ylabel("País")
ax.xaxis.set_major_formatter("{x:,.0f}")

# Anotación del valor de cada barra
total = paises2.Cantidad.sum()
for barra in ax.patches:
    h, w = barra.get_y(), barra.get_width()
    ax.text(x = w + 1500, y = h + 0.38, s = "{:.2%}".format(w/total), # "{:,.0f}".format(w)
            ha = 'center',
            va = 'center')

In [ ]:
fig = px.bar(paises2, x = "Cantidad", y = "country")
fig.show()

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure([
    go.Bar(x = paises2["country"], y = paises2["Cantidad"])
    ])
fig.show()

In [ ]:
# https://plotly.com/python/horizontal-bar-charts/
fig = go.Figure([
    go.Bar(y = paises2["country"], x = paises2["Cantidad"], orientation = 'h', 
           marker = dict(color = 'rgba(50, 171, 96, 0.6)', line = dict(color = 'rgba(50, 171, 96, 1.0)', width = 1)),
           hovertemplate = 'País: %{y}'+ '<br>Cantidad: %{x}'
           )
    ])

fig.update_layout(
    title = 'Tipos de vino por país',
    yaxis = dict(
        showgrid = False,
        showticklabels = True,
    ),
    xaxis = dict(
        showticklabels = True,
        showgrid = False
    ),
    paper_bgcolor = 'rgb(248, 248, 255)',
    plot_bgcolor = 'rgb(248, 248, 255)',
)

fig.show()

In [ ]:
# Histograma de la puntuación de los vinos combinando grafico directo de pandas y Matplotlib para formato
fig, ax = plt.subplots(figsize = (20, 6))

sns.histplot(data = vinos, x = "points", bins = 20, ax = ax)
plt.xlabel('Puntuación')
plt.ylabel('Frecuencia')
plt.title('Distribución de la puntuación de vinos')

# Linea vertical en el umbral de recomendación
plt.axvline(x = 92, linewidth = 2, color = 'r', linestyle = "dashed")
plt.show()

In [ ]:
fig = px.histogram(vinos, x = "points", nbins = 20, opacity = 0.8)
fig.show()

In [ ]:
vinos_paises = vinos.loc[vinos.country.isin(["Italy", "France"]),].reset_index(drop = True)
vinos_paises["country"] = vinos_paises["country"].astype(str)

# Histograma de la puntuación de los vinos combinando grafico directo de pandas y Matplotlib para formato
fig, ax = plt.subplots(figsize = (20, 6))

sns.histplot(data = vinos_paises, x = "points", hue = "country", stat = "density", bins = 10, 
             palette = ["green", "blue"], ax = ax)
plt.xlabel('Puntuación')
plt.ylabel('Frecuencia')
plt.title('Distribución de la puntuación de vinos')

# Linea vertical en el umbral de recomendación
plt.axvline(x = 92, linewidth = 2, color = 'r', linestyle = "dashed")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (20, 6))

sns.boxplot(data = vinos, x = "class", y = "price", ax = ax, palette = "RdYlGn")
plt.ylim(0, 500)
plt.xlabel('Calidad del vino')
plt.ylabel('Precio del vino')
ax.yaxis.set_major_formatter("${x:,.0f}")

## TAREA - Taller 

 1. Además de entender las posibles distribuciones o gráficos de dispersión a través del tiempo, nos interesa poder visualizar las relaciones entre dos variables numéricas. Esto nos permite observar si existe algun tipo de correlacion. Investigue e implemente el comando `plt.scatter` o `sns.scatter` para evidenciar cualesquiera dos de sus variables. ¿Qué historia se puede contar?

2. Use un gráfico de torta (un pie graph en inglés) para ver la distribución por clase. Investigue si existe alguna forma de "extraer" (explode en inglés) pedazos de dicha torta 

3. Incorpore las 2 vistas para que estén en un único visual.